In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D, Concatenate, Input, AveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications  
from tensorflow.keras.applications import ResNet50
from keras.models import Model


In [ ]:
tune_model = applications.vgg16.VGG16(weights='imagenet', include_top=False,input_shape=(96, 96, 3))
model = Sequential()
for layer in tune_model.layers:
    model.add(layer)
for layer in model.layers[2:]:
    layer.trainable = False

model.add(Flatten())
model.add(Dense(512,activation='elu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='elu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='elu'))
model.add(Dropout(0.3))

model.add(Dense(2,activation='softmax'))


model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy']
            )
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

In [ ]:
r1 = model.fit(images, labels, epochs=10, validation_split=0.2, batch_size=32, callbacks=[early_stop])

In [ ]:
results = model.evaluate(X_test, Y_test, batch_size=32)
print(f"Loss: {results[0]}  Accuracy: {results[1]}")

In [ ]:
model.save("SAS.keras")